# Imports

In [1]:
import pandas as pd
import numpy as np
import keras
import math

Using TensorFlow backend.


# Load the dataset

In [18]:
# one hot encode the dataset, only take 6:00, 12:00 and 18:00 b/c that when most 
# of the times are taken
dataframe = pd.read_csv("atlantic_without_NW.csv")
dataframe["time"] = dataframe["time"].astype('category')
dataframe['date'] = [str(i)[4:6] for i in dataframe['date']] # Get just the month

dataframe = pd.get_dummies(dataframe, columns=["time","rating","date"])
dataframe.drop(["ocean","name"],axis=1, inplace=True)

y = dataframe[["rating_ DB","rating_ EX","rating_ HU","rating_ LO","rating_ SD","rating_ SS","rating_ TD","rating_ TS","rating_ V"]].values

X = dataframe[["lat","long","wind_speed","pressure","time_600","time_1200","time_1800","date_01","date_04","date_05","date_06","date_07","date_08","date_09","date_10","date_11","date_12"]].values



# Clean the data

In [19]:
# delete all the rows that contain no readings for pressure
rows_to_delete = []
row_num = 0

for row in X:
    if row[3] == -999:
        rows_to_delete.append(row_num)
    row_num += 1

X = np.delete(X, rows_to_delete, 0)        # remove all rows with bad data
y = np.delete(y, rows_to_delete, 0)

encoded_lat_long = np.zeros(shape=(len(X),3))
count = 0
R = 6371 # radius of earth
for row in X:
    lat = np.deg2rad(row[0])
    lng = np.deg2rad(row[1])

    encoded_lat_long[count][0] = R * (math.cos(lat) * math.cos(lng))
    encoded_lat_long[count][1] = R * (math.cos(lat) * math.sin(lng))
    encoded_lat_long[count][2] = R * math.sin(lat)
    count += 1

X = np.delete(X, 0, 1)                     # remove old lat
X = np.delete(X, 0, 1)                     # remove old long
X = np.append(X, encoded_lat_long, axis= 1)


In [20]:
print(X[0])

[2.50000000e+01 1.00900000e+03 0.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 3.46581665e+03
 4.98665664e+03 1.92639857e+03]


# Split the dataset into training set and test set

In [21]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)


# Train the model on training set

In [6]:
from keras.models import Sequential
from keras.layers import Dense

In [22]:
classifier = Sequential()

# input layer
classifier.add(Dense(output_dim=9, init='uniform', activation='relu', input_dim=18))

# hidden layer
classifier.add(Dense(output_dim=10, init='uniform', activation='relu'))

# hidden layer
classifier.add(Dense(output_dim=14, init='uniform', activation='relu'))

# hidden layer
classifier.add(Dense(output_dim=16, init='uniform', activation='relu'))

# output layer
classifier.add(Dense(output_dim=9, init='uniform', activation='softmax'))

# create the NN
classifier.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

# Predict the test set


In [13]:

classifier.fit(x=X_train, y=y_train, validation_data=(X_test,y_test), batch_size=10, epochs=200)

=====] - 3s 279us/step - loss: 0.6673 - accuracy: 0.7809 - val_loss: 0.6136 - val_accuracy: 0.8000
Epoch 69/200
12380/12380 [==============================] - 3s 274us/step - loss: 0.6687 - accuracy: 0.7809 - val_loss: 0.6124 - val_accuracy: 0.8000
Epoch 70/200
12380/12380 [==============================] - 3s 270us/step - loss: 0.6629 - accuracy: 0.7811 - val_loss: 0.6101 - val_accuracy: 0.8000
Epoch 71/200
12380/12380 [==============================] - 3s 259us/step - loss: 0.6652 - accuracy: 0.7814 - val_loss: 0.6109 - val_accuracy: 0.8000
Epoch 72/200
12380/12380 [==============================] - 3s 264us/step - loss: 0.6620 - accuracy: 0.7821 - val_loss: 0.6188 - val_accuracy: 0.8000
Epoch 73/200
12380/12380 [==============================] - 3s 272us/step - loss: 0.6722 - accuracy: 0.7808 - val_loss: 0.6093 - val_accuracy: 0.7994
Epoch 74/200
12380/12380 [==============================] - 3s 266us/step - loss: 0.6585 - accuracy: 0.7829 - val_loss: 0.6143 - val_accuracy: 0.8000
E

In [9]:
GH = np.array([[5,1092,0,1,0]])

y_pred = classifier.predict(GH)

ValueError: Error when checking input: expected dense_1_input to have shape (18,) but got array with shape (5,)

In [10]:
for row in y_pred:
    most_likely = 0
    for el in row:
        if el > most_likely:
            most_likely = el

    print( "rating_DB: {DB} \nrating_EX: {EX}\nrating_HU: {HU} \nrating_LO: {LO} \nrating_SD: {SD} \nrating_SS: {SS} \nrating_TD: {TD} \nrating_TS: {TS} \nrating_WV: {WV} \n\nMost Likely: {ml}\n\n".format(DB=row[0], EX=row[1], HU=row[2], LO=row[3], SD=row[4], SS=row[5], TD=row[6], TS=row[7], WV=row[8], ml=most_likely) )

NameError: name 'y_pred' is not defined